In [1]:
# import all the required libraries
import requests
from requests_oauthlib import OAuth1
import pandas as pd
from pandas import json_normalize
from pandas import DataFrame
import json
import time

In [7]:
# get your consumer_key and consumer_secret from your twitter developer account
consumer_key = "XXXXXXXXXXXXXXXXXXXX" 
consumer_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [36]:
# type your keywords, you want to use to extract the tweets
q = '#USElections2020 OR #USElections OR #2020Elections OR #2020Election OR #USPolitics OR #Elections2020 OR #US OR \
#Elections OR elections OR trump OR moscowmitch OR democrats OR #maga OR donaldtrump OR #voteblue OR bluewave OR \
biden OR bernie OR petebuttigieg OR republican OR liberal OR #berniesanders OR kag OR democrat OR #usa OR andrewyang \
OR #impotus  OR mikepence OR #trumpsucks OR #yanggang OR qanon OR gop OR pete -filter:retweets AND -filter:replies'

# url to extract tweets
url = 'https://api.Twitter.com/1.1/search/tweets.json'

# set your parameters
# maximum of 100 tweets can be extracted in one call
pms = {'q':q,'count':100, 'lang':'en', 'tweet_mode':'extended'}

# for authentication
auth = OAuth1(consumer_key,consumer_secret)


In [ ]:
# initiate your dataframe to store the tweets
tweets_df = pd.DataFrame()

In [47]:
# initiate the page_counter
pages_counter = 0

# total number of tweets to be extracted, 100*number_of_pages
number_of_pages = 2000

#start your loop for 2000 calls
while pages_counter<number_of_pages:
    pages_counter += 1
    res = requests.get(url,params = pms,auth = auth)
    
    # print the connection status and call number
    print("Connection Status: %s, page =  " % res.reason,pages_counter )
    
    # get a json file of the extracted tweets
    tweets = res.json()
    # convert to dataframe
    df = json_normalize(tweets['statuses'])
    
    # set the pms id so that same tweets are not extracted in every call
    ids = [i['id'] for i in tweets['statuses']]
    pms['max_id'] = min(ids)-1
    
    # add the extracted tweets to our initial dataframe
    tweets_df = pd.concat([tweets_df, df], ignore_index=True)

Connection Status: OK, page =   1
Connection Status: OK, page =   2
Connection Status: OK, page =   3
Connection Status: OK, page =   4
Connection Status: OK, page =   5
Connection Status: OK, page =   6
Connection Status: OK, page =   7
Connection Status: OK, page =   8
Connection Status: OK, page =   9
Connection Status: OK, page =   10
Connection Status: OK, page =   11
Connection Status: OK, page =   12
Connection Status: OK, page =   13
Connection Status: OK, page =   14
Connection Status: OK, page =   15
Connection Status: OK, page =   16
Connection Status: OK, page =   17
Connection Status: OK, page =   18
Connection Status: OK, page =   19
Connection Status: OK, page =   20
Connection Status: OK, page =   21
Connection Status: OK, page =   22
Connection Status: OK, page =   23
Connection Status: OK, page =   24
Connection Status: OK, page =   25
Connection Status: OK, page =   26
Connection Status: OK, page =   27
Connection Status: OK, page =   28
Connection Status: OK, page =

ConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/search/tweets.json?q=%23USElections2020+OR+%23USElections+OR+%232020Elections+OR+%232020Election+OR+%23USPolitics+OR+%23Elections2020+OR+%23US+OR+%23Elections+OR+elections+OR+trump+OR+moscowmitch+OR+democrats+OR+%23maga+OR+donaldtrump+OR+%23voteblue+OR+bluewave+OR+biden+OR+bernie+OR+petebuttigieg+OR+republican+OR+liberal+OR+%23berniesanders+OR+kag+OR+democrat+OR+%23usa+OR+andrewyang+OR+%23impotus++OR+mikepence+OR+%23trumpsucks+OR+%23yanggang+OR+qanon+OR+gop+OR+pete+-filter%3Aretweets+AND+-filter%3Areplies&count=100&lang=en&tweet_mode=extended&max_id=1265004395571412997 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001E0C3BC0908>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected

In [48]:
# number of tweets and columns
tweets_df.shape

(215231, 187)

In [50]:
# filter the useful columns
tweets_df = tweets_df.filter(items = ['created_at','id','full_text','source','user.id','user.name','user.location','user.followers_count','user.lang'])

In [52]:
# incase of any duplicate tweets, drop the duplicate tweets 
tweets_df = tweets_df.drop_duplicates(subset = ['created_at','id'])
tweets_df.shape

(215231, 9)

In [54]:
# store 
tweets_df.to_csv('us_elections2020.csv',index=False, header = True)